In [ ]:
!pip install tensorflow opencv-python matplotlib


In [8]:
#Import standard dependcies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt


In [9]:
#Import tensorflow dependcies
from tensorflow.keras.models import Model
#layer allows us build a custom layer
#Conv2D allows us to make convulional neural networks
#MaxPooling reduces the amount of data we pass to the next layer
#Input allows us to pass through to our model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [10]:
#Sets up folder path's
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [ ]:
#MAKES DIRECTORIES
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [1]:
#Uncompressed the labeled faces in the wild data set
#This file is used to validate against our face (aka our "negative images")
!tar -xf lfw.tgz

In [6]:
#adds all files to the negative directory
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

In [4]:
#This librarys allows us to generate uniqe images names
import uuid

In [6]:
#Creates a connection to the webcam
cap = cv2.VideoCapture(0)
#loops through every frame
while cap.isOpened():
    #captures the frame
    ret, frame = cap.read()
    #sets the frame to be 250 by 250
    frame = frame[120:120+250,200:200+250, :]
    
    #Collects anchor images when a is pressed
    if cv2.waitKey(1) & 0xFF == ord('a'):
        #Creates a uniqe image name
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        #saves the frame to the directory
        cv2.imwrite(imgname, frame)
    #Collects a positive image
    if cv2.waitKey(1) & 0xFF == ord('p'):
         #Creates a uniqe image name
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        #saves the frame to the directory
        cv2.imwrite(imgname, frame)
    
    #shows the image onto the screen
    cv2.imshow('Image Collection', frame)
    #Stops the function
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#For some reason in Mac, this doesn't work
#will look more deeply into it later
cv2.destroyWindow('Image Collection')

#releases the webcam
cap.release()

In [22]:
#takes 300 images of each type of image collected into an tf dataset
anchor = tf.data.Dataset.list_files(ANC_PATH+'/*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH+'/*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH+'/*.jpg').take(300)

In [27]:
#prepoccess the images
def preprocess(file_path):
    #Read the image from file path
    byte_img = tf.io.read_file(file_path)
    #loads the iamge
    img = tf.io.decode_jpeg(byte_img)
    #resizes
    img = tf.image.resize(img,(100,100))
    #re-scales
    img = img / 255.0
    return img

In [29]:
#creates labeled positives(represented by 1)
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
#Creates labeled negatives(represented by 0)
negaitves = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negaitves)

In [35]:
#returned loaded and preprocessed image
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [36]:
# Builds the data loader pipeline
data = data.map(preprocess_twin)
data = data.cache()
#ensures images are mixed instead of all positives
#followed by all negatives
data = data.shuffle(buffer_size=1024)

In [37]:
data

<_ShuffleDataset element_spec=(TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [40]:
#Training parition of data
train_data = data.take(round(len(data) * .7))
train_data = train_data.batch(16)
#prefecteches images do we don't overload our neural network
train_data = train_data.prefetch(8)

In [41]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [43]:
#Testing parition of data
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)